In [1]:
import pandas as pd

We print log for commits between May 1, 2021(including) and May 31, 2021(including) that contain functional changes(commit message does not contain "doc")

In [2]:
db_pwd = "~/VictoriaMetrics/"
pwd_output = !pwd
current_dir = pwd_output[0]
filter_params = '--since "FEBRUARY 28 2021" --until "JUNE 1 2021" --pretty=format:"%h; %cd; %s"  --grep="(#" --regexp-ignore-case'


In [3]:
!cd {db_pwd} && git log --oneline {filter_params} | wc -l

69


We have 86 commits that contain functional changes, pull request are squashed before merge, so we do not need to exclude something else.

Let's save these commits as commits.csv

In [4]:
!cd {db_pwd} && git log {filter_params} > {current_dir}/commits.csv

Process data and format it to `number (recent commits at the top); version1 hash (new); version2 hash (old); date (optional, for debugging); subject (optional, for debugging)`

In [5]:
df = pd.read_csv('commits.csv', sep=';', names=["hash", "date", "message"])
df = df.reindex(index=df.index[::-1])
df.reset_index(inplace=True, drop=True)
df = df.reindex(index=df.index[::-1])
df.head()

,hash,date,message
69,ddc80227,Mon May 31 09:21:23 2021 +0300,fixes solaris build (#1345)
68,f0c21b63,Thu May 27 14:52:44 2021 +0300,lib/promscrape: apply body size & sample limi...
67,2bbb1cc7,Wed May 26 12:28:58 2021 +0300,Docs review (#1330)
66,7ecaa2fe,Wed May 26 12:26:45 2021 +0300,update the issue template (#1329)
65,d210958f,Tue May 25 14:27:22 2021 +0100,vmalert: automatically reload configuration o...


In [6]:
df['new_version'] = df.hash.copy()
df['baseline'] = df.hash[0]
df = df.drop(df.index[-1])
df.tail()

,hash,date,message,new_version,baseline
5,433fff00,Tue Mar 9 12:26:23 2021 +0200,duplicate timeseries fix for prometheus_bucke...,433fff00,f6114345
4,364fdf4a,Tue Mar 9 09:18:19 2021 +0200,spelling fix: adjacent (#1115),364fdf4a,f6114345
3,621bf037,Wed Mar 3 10:51:51 2021 +0200,Vmagent docs upd (#1104),621bf037,f6114345
2,e154f4a6,Wed Mar 3 10:43:39 2021 +0200,Fix ingress discovery api (#1110),e154f4a6,f6114345
1,f4f8f218,Wed Mar 3 07:46:10 2021 +0000,vmalert: docs update - status endpoint needs ...,f4f8f218,f6114345


In [7]:
df = df[['new_version', 'baseline', 'date', 'message']]
df.head()

,new_version,baseline,date,message
69,ddc80227,f6114345,Mon May 31 09:21:23 2021 +0300,fixes solaris build (#1345)
68,f0c21b63,f6114345,Thu May 27 14:52:44 2021 +0300,lib/promscrape: apply body size & sample limi...
67,2bbb1cc7,f6114345,Wed May 26 12:28:58 2021 +0300,Docs review (#1330)
66,7ecaa2fe,f6114345,Wed May 26 12:26:45 2021 +0300,update the issue template (#1329)
65,d210958f,f6114345,Tue May 25 14:27:22 2021 +0100,vmalert: automatically reload configuration o...


In [8]:
df.to_csv('commitTable.csv',index_label='number',sep=';')
df = df.sample(n=5, random_state=123)
df.to_csv('commitTableTest.csv',index_label='number',sep=';')